# Test Pipeline from notebook
- here I try to use current DC modules from notebook, as python lib (fails).

In [1]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import csv
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn import datasets

In [2]:
data_path = "data/movies/original/"
file_a = data_path+"roger_ebert.csv"
file_b = data_path+"imdb.csv"

df_a_original = pd.read_csv(file_a, encoding='utf-8')
df_b_original = pd.read_csv(file_b, encoding='utf-8')

## Run Fahes
- Discover DMVs
- Set DMVs to *Null*

In [4]:
## Fahes
from services.fahes_service import fahes_api
fahes_path_in = "data/movies/fahes/in/"
fahes_path_out = "data/movies/fahes/out/"

print(df_a_original.year.isnull().sum())
#df_a_original = df_a_original.dropna()
#df_a_original.year = df_a_original.year.astype(int)
#print(df_a_original.columns)

## random inject DMVs
X = np.random.randint(0, len(df_a_original), size=99)
for x in X:
    df_a_original.loc[x,'year'] = '?'

## save as csv
df_a_original.to_csv(fahes_path_in+"data_a.csv",encoding="utf-8",sep=',',index=False, quoting = csv.QUOTE_NONNUMERIC, header=True)
df_b_original.to_csv(fahes_path_in+"data_b.csv",encoding="utf-8",sep=',',index=False, quoting = csv.QUOTE_NONNUMERIC, header=True)

folder_in_json = {'CSV':{'dir': fahes_path_in, 'table':''}}
folder_out_json = {'CSV':{'dir': fahes_path_out}}

fahes_api.executeService(folder_in_json, folder_out_json)

5


In [5]:
pd.read_csv(fahes_path_out+"data_a.csv", encoding='utf-8').year.isnull().sum()

197

In [6]:
with open(fahes_path_in+"data_a.csv", 'U') as f:
 
    newText=f.read()
 
    while '.0' in newText:
        newText=newText.replace('.0', '')
 
    with open(fahes_path_in+"data_a.csv", "w") as f:
        f.write(newText)

with open(fahes_path_out+"data_a.csv", 'U') as f:
 
    newText=f.read()
 
    while '.0' in newText:
        newText=newText.replace('.0', '')
 
    with open(fahes_path_out+"data_a.csv", "w") as f:
        f.write(newText)

In [7]:
#from services.tmp import fahes_api
from services.impute import imputedb_api

In [12]:
#imputedb_api.executeService(input_source, output_destination, q, r)
imputedb_api.executeService({"CSV":{"dir":fahes_path_out, "table":"data_a"}}, 
                            {"CSV":{"dir":"data/movies/impute/out/"}}, 
                            "select year from data_a;",0)

CalledProcessError: Command '['/Users/gio/Workspace/dcapp2/services/impute/imputedb/imputedb', 'load', '--db', '/Users/gio/Workspace/dcapp2/services/impute/app/storage/imputedb/tmp.db', 'data/movies/fahes/out//data_a.csv']' returned non-zero exit status 1

In [22]:
# Use only common columns
cols = [u'movie_name', u'genre', u'directors', u'actors', u'duration', u'id']
df_a = df_a_original[cols]
df_b = df_b_original[cols]

## labeled data
file_l = data_path+"labeled_data.csv"
df_l = pd.read_csv(file_l, encoding='utf-8')
df_l = df_l[["ltable.id","rtable.id","gold"]]
df_l.columns = [["ltable_id","rtable_id","gold"]]

## candidate pairs (from blocking)
file_c = data_path+"candset.csv"
df_c = pd.read_csv(file_c, encoding='utf-8')
df_c = df_c[["ltable.id","rtable.id"]]
df_c.columns = [["ltable_id","rtable_id"]]
df_c["gold"] = 0